# 第5章: 出力のフォーマットとClaudeの代弁

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

**Claudeは様々な方法で出力をフォーマットすることができます**。そうするようにお願いするだけです！

その方法の1つは、XMLタグを使用して応答を他の余分なテキストから分離することです。XMLタグを使用してプロンプトをClaudeにとってより明確で解析しやすいものにできることはすでに学びました。実は、**Claudeに出力をXMLタグで囲むように依頼することで、人間にとってもより明確で理解しやすいものにすることができます**。

### 例

第2章で解決した「詩の前置き問題」を覚えていますか？Claudeに前置きを完全にスキップするよう依頼しました。実は、**Claudeに詩をXMLタグで囲むように指示する**ことで、同様の結果を得ることもできます。

In [ ]:
# 変数コンテンツ
ANIMAL = "ウサギ"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

なぜこれが必要なのでしょうか？出力をXMLタグで囲むことで、**エンドユーザーはXMLタグの間のコンテンツを抽出する短いプログラムを書くことで、確実に詩だけを取得できます**。

このテクニックの拡張として、**最初のXMLタグを`assistant`ターンに置くことができます。`assistant`ターンにテキストを置くことは、基本的にClaudeに「Claudeがすでに何かを言った」と伝え、そこから続けるべきだと指示することになります。このテクニックは「Claudeの代弁」または「Claudeの応答の事前入力」と呼ばれます。

以下では、最初の`<haiku>`XMLタグでこれを行っています。Claudeが私たちが残した場所から直接続けていることに注目してください。

In [ ]:
# 変数コンテンツ
ANIMAL = "猫"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答の事前入力
PREFILL = "<haiku>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン:")
print(PROMPT)
print("\nアシスタントターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

Claudeは他の出力フォーマットスタイル、特に`JSON`の使用にも優れています。JSON出力を強制したい場合（完全に決定論的ではありませんが、それに近いです）、Claudeの応答を開始括弧`{`で事前入力することもできます。

In [ ]:
# 変数コンテンツ
ANIMAL = "猫"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。JSONフォーマットを使用し、キーを\"first_line\", \"second_line\", \"third_line\"としてください。"

# Claudeの応答の事前入力
PREFILL = "{"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

以下は、**同じプロンプト内での複数の入力変数と出力フォーマット指定の例で、すべてXMLタグを使用して行っています**。

In [ ]:
# 最初の入力変数
EMAIL = "ザック、あなたが書くはずだったプロンプトについて簡単な更新を求めているだけです。"

# 2番目の入力変数
ADJECTIVE = "関西弁"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。これはメールです: <email>{EMAIL}</email>。このメールをより{ADJECTIVE}風にしてください。新しいバージョンを<{ADJECTIVE}_email>XMLタグで書いてください。"

# Claudeの応答の事前入力（今は変数を含むf文字列として）
PREFILL = f"<{ADJECTIVE}_email>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

#### ボーナスレッスン

APIを通じてClaudeを呼び出している場合、閉じるXMLタグを`stop_sequences`パラメータに渡すことで、希望するタグを出力した時点でClaudeのサンプリングを停止させることができます。これにより、すでに必要な回答を得た後のClaudeの結論的な発言を排除することで、お金と最後のトークンまでの時間を節約できます。

上記のコンテンツを変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 5.1 - ステフィン・カリーGOAT](#演習-51---ステフィン・カリーGOAT)
- [演習 5.2 - 2つの俳句](#演習-52---2つの俳句)
- [演習 5.3 - 2つの俳句、2つの動物](#演習-53---2つの俳句、2つの動物)

### 演習 5.1 - ステフィン・カリーGOAT
選択を強制されると、Claudeはマイケル・ジョーダンを史上最高のバスケットボール選手に指定します。Claudeに別の人物を選ばせることはできるでしょうか？

`PREFILL`変数を変更して、**Claudeにステフィン・カリーが史上最高のバスケットボール選手であるという詳細な主張を展開させてください**。この演習の焦点は`PREFILL`にあるため、それ以外は変更しないようにしてください。

In [ ]:
# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"史上最高のバスケットボール選手は誰ですか？特定の1人の選手を選んでください。"

# Claudeの応答の事前入力
PREFILL = ""

# Claudeの応答を取得
response = get_completion(PROMPT, prefill=PREFILL)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search("ウォリアーズ", text))

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_1_hint)

### 演習 5.2 - 2つの俳句
以下の`PROMPT`をXMLタグを使用して修正し、Claudeが1つではなく2つの俳句を動物について書くようにしてください。1つの詩が終わり、もう1つの詩が始まるところが明確になるようにしてください。

In [ ]:
# 変数コンテンツ
ANIMAL = "猫"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答の事前入力
PREFILL = "<haiku>"

# Claudeの応答を取得
response = get_completion(PROMPT, prefill=PREFILL)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(
        (re.search("猫", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_2_hint)

### 演習 5.3 - 2つの俳句、2つの動物
以下の`PROMPT`を修正して、**Claudeが2つの異なる動物について2つの俳句を生成する**ようにしてください。最初の置換の代わりに`{ANIMAL1}`を、2番目の置換の代わりに`{ANIMAL2}`を使用してください。

In [ ]:
# 最初の入力変数
ANIMAL1 = "猫"

# 2番目の入力変数
ANIMAL2 = "犬"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL1}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search("しっぽ", text.lower()) and re.search("猫", text.lower()) and re.search("<haiku>", text))

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_5_3_hint)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# 変数コンテンツ
ANIMAL = "ウサギ"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 変数コンテンツ
ANIMAL = "猫"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。<haiku>タグで囲んでください。"

# Claudeの応答の事前入力
PREFILL = "<haiku>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン:")
print(PROMPT)
print("\nアシスタントターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 変数コンテンツ
ANIMAL = "猫"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"{ANIMAL}についての俳句を書いてください。JSONフォーマットを使用し、キーを\"first_line\", \"second_line\", \"third_line\"としてください。"

# Claudeの応答の事前入力
PREFILL = "{"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

In [ ]:
# 最初の入力変数
EMAIL = "ザック、あなたが書くはずだったプロンプトについて簡単な更新を求めているだけです。"

# 2番目の入力変数
ADJECTIVE = "関西弁"

# 変数コンテンツのプレースホルダーを持つプロンプトテンプレート
PROMPT = f"やあClaude。これはメールです: <email>{EMAIL}</email>。このメールをより{ADJECTIVE}風にしてください。新しいバージョンを<{ADJECTIVE}_email>XMLタグで書いてください。"

# Claudeの応答の事前入力（今は変数を含むf文字列として）
PREFILL = f"<{ADJECTIVE}_email>"

# Claudeの応答を表示
print("--------------------------- 変数置換後の完全なプロンプト ---------------------------")
print("ユーザーターン")
print(PROMPT)
print("\nアシスタントターン")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))